# Various examples using the NNDC query tools

First, import necessary tools.

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.width', 200)
from becquerel.tools import nndc

## Example 1: Cesium-137

### 1a: Look up the half-life of Cesium-137
Fetch the "wallet card" data for Cs-137. The wallet card includes the Z, A, half-life, nuclear spin, etc.

In [25]:
card = nndc.fetch_wallet_card(nuc='Cs-137')
print(card)
print('')
t12 = max(card['T1/2 (s)'])
print('{} s'.format(t12))
print('{} y'.format(t12 / 3600 / 24 / 365.25))

    Z Element    A m  M   N   JPi  Energy Level (MeV) Decay Mode  Branching (%) Abundance (%) Mass Excess (MeV)     T1/2 (s)    T1/2 (txt)
0  55      Cs  137    0  82  7/2+                 0.0         B-          100.0          None           -87+/-4  949300000.0  30.08 Y 0.09

949300000.0 s
30.08150176185768 y


### 1b: Find the branching ratio of the 662 keV gamma-ray line of Cs-137
Fetch the "decay radiation" data for the line at 662 keV. For gamma rays, the ```type``` keyword should be set to ```"Gamma"```.

In [24]:
rad = nndc.fetch_decay_radiation(nuc='Cs-137', type='Gamma',
                                 i_range=(1, None), e_range=(661, 663))
print(rad)
print('')
print('{:.2f} keV line intensity: {}%'.format(
    rad['Radiation Energy (keV)'][0],
    rad['Radiation Intensity (%)'][0]))

    Z Element    A   N   JPi Decay Mode Radiation Radiation subtype Radiation Energy (keV) Radiation Intensity (%) Daughter Dose (MeV / Bq / s) Endpoint Energy (keV)  Parent Energy Level (MeV)  \
0  55      CS  137  82  7/2+         B-         G                        661.6570+/-0.0030            85.10+/-0.20    137BA     0.5631+/-0.0013                  None                        0.0   

      T1/2 (s) T1/2 (txt)  
0  949300000.0  30.08 Y 9  

661.66+/-0.00 keV line intensity: 85.10+/-0.20%


## Example 2: Barium-133

### 2a: Fetch isotope data for Ba-133

In [4]:
card = nndc.fetch_wallet_card(nuc='Ba-133')
print(card)

    Z Element    A  m  M   N    JPi  Energy Level (MeV) Decay Mode  Branching (%) Abundance (%) Mass Excess (MeV)     T1/2 (s)      T1/2 (txt)
0  56      Ba  133     0  77   1/2+                 0.0         EC         100.00          None          -88+/-10  333000000.0  10.551 Y 0.011
1  56      Ba  133  m  1  77  11/2-               288.3         IT          99.99          None          -87+/-10     140148.0     38.93 H 0.1
2  56      Ba  133  m  1  77  11/2-               288.3         EC           0.01          None          -87+/-10     140148.0     38.93 H 0.1


The basic query includes the 288 MeV excited state with a half-life of 39 hours. By using the ```elevel_range``` keyword we can query only the ground state:

In [5]:
card = nndc.fetch_wallet_card(nuc='Ba-133', elevel_range=(0, 0))
print(card)

    Z Element    A m  M   N   JPi  Energy Level (MeV) Decay Mode  Branching (%) Abundance (%) Mass Excess (MeV)     T1/2 (s)      T1/2 (txt)
0  56      Ba  133    0  77  1/2+                 0.0         EC          100.0          None          -88+/-10  333000000.0  10.551 Y 0.011


### 2b: Find the main gamma-ray lines of Ba-133
Next, find all gamma-rays for decays from the ground state of Ba-133 that are &ge;5% intensity. Use the ```i_range``` keyword to select a range of intensities.

In [6]:
rad = nndc.fetch_decay_radiation(nuc='Ba-133', type='Gamma', i_range=(5, None))
cols = ['Z', 'Element', 'A', 'Decay Mode', 'Radiation', 'Radiation Energy (keV)',
        'Radiation Intensity (%)', 'Parent Energy Level (MeV)']
print(rad[cols])

     Z Element    A Decay Mode Radiation Radiation Energy (keV) Radiation Intensity (%) Parent Energy Level (MeV)
0   56      BA  133         IT         G                   4.47              16.4+/-0.8           288.252+/-0.009
1   56      BA  133         IT         G                 31.817              15.1+/-0.4           288.252+/-0.009
2   56      BA  133         IT         G                 32.194              27.6+/-0.6           288.252+/-0.009
3   56      BA  133         IT         G                 36.378             5.10+/-0.11           288.252+/-0.009
4   56      BA  133         IT         G        275.925+/-0.007                   17.69           288.252+/-0.009
5   56      BA  133         EC         G                   4.29              15.7+/-0.8                         0
6   56      BA  133         EC         G                 30.625              33.9+/-1.0                         0
7   56      BA  133         EC         G                 30.973              62.2+/-1.8 

Once again, this query includes the short-lived excited state. For decay radiation queries, there is no ```elevel_range``` keyword available, so the ground state must be selected from the returned DataFrame:

In [7]:
rad = rad[rad['Parent Energy Level (MeV)'] == 0]  # ground state only
print(rad[cols])

     Z Element    A Decay Mode Radiation Radiation Energy (keV) Radiation Intensity (%) Parent Energy Level (MeV)
5   56      BA  133         EC         G                   4.29              15.7+/-0.8                         0
6   56      BA  133         EC         G                 30.625              33.9+/-1.0                         0
7   56      BA  133         EC         G                 30.973              62.2+/-1.8                         0
8   56      BA  133         EC         G                  34.92             5.88+/-0.17                         0
9   56      BA  133         EC         G                 34.987            11.40+/-0.30                         0
10  56      BA  133         EC         G       80.9979+/-0.0011            32.90+/-0.30                         0
11  56      BA  133         EC         G      276.3989+/-0.0012             7.16+/-0.05                         0
12  56      BA  133         EC         G      302.8508+/-0.0005            18.34+/-0.13 

## Example 3: Hf-178 (multiple excited states)

Look up the half-lives for the ground state and two excited states of Hf-178:

In [8]:
card = nndc.fetch_wallet_card(nuc='Hf-178')
print(card)

    Z Element    A   m  M    N  JPi  Energy Level (MeV) Decay Mode  Branching (%) Abundance (%) Mass Excess (MeV)      T1/2 (s) T1/2 (txt)
0  72      Hf  178      0  106   0+                 0.0                       NaN  27.28+/-0.07          -52+/-21           inf     STABLE
1  72      Hf  178  m1  1  106   8-              1147.4         IT          100.0          None          -51+/-21  4.000000e+00  4.0 S 0.2
2  72      Hf  178  m2  2  106  16+              2446.1         IT          100.0          None          -50+/-21  9.783000e+08     31 Y 1


## Example 4: Uranium-235

### 4a: Find the different types of decays of U-235
This information is in the wallet card:

In [9]:
card = nndc.fetch_wallet_card(nuc='U-235', elevel_range=(0, 0))
print(set(card['Decay Mode']))

{'28Mg', 'SF', 'Ne', 'A'}


### 4b: Find the main alpha lines for U-235

In [10]:
rad = nndc.fetch_decay_radiation(nuc='U-235', type='Alpha', i_range=(10, None))
print(rad[cols])

    Z Element    A Decay Mode Radiation Radiation Energy (keV) Radiation Intensity (%)  Parent Energy Level (MeV)
0  92       U  235          A         A           4364.3+/-0.4            18.92+/-0.14                        0.0
1  92       U  235          A         A           4395.4+/-0.4            57.73+/-0.24                        0.0


### 4c: Find the main gamma lines for U-235

In [11]:
rad = nndc.fetch_decay_radiation(nuc='U-235', type='Gamma', i_range=(10, None))
print(rad[cols])

    Z Element    A Decay Mode Radiation Radiation Energy (keV) Radiation Intensity (%)  Parent Energy Level (MeV)
0  92       U  235          A         G                     13                  27+/-4                        0.0
1  92       U  235          A         G           19.55+/-0.05                      63                        0.0
2  92       U  235          A         G        143.760+/-0.020            10.96+/-0.14                        0.0
3  92       U  235          A         G        185.715+/-0.005              57.0+/-0.6                        0.0


## Example 5: Pa-234m

### Look up the 1001 keV gamma-ray line from the decay of Pa-234m
Requires searching for decays from an excited state of a nucleus. First query the decay radiation for the isotope, then select on the ```"Parent Energy Level (MeV)"```.

In [12]:
rad = nndc.fetch_decay_radiation(nuc='Pa-234', type='Gamma', e_range=(999, 1002))
rad = rad[rad['Parent Energy Level (MeV)'] > 0]  # excited state ~73 MeV
print(rad[cols])

    Z Element    A Decay Mode Radiation Radiation Energy (keV) Radiation Intensity (%)  Parent Energy Level (MeV)
0  91      PA  234         B-         G         1001.03+/-0.10           0.842+/-0.008                      73.92


## Example 6: Find long-lived isotopes with gamma-ray lines at 1461 keV
Are there any others besides K-40? Use the ```t_range``` keyword to look for any other isotopes with half-lives greater than 1E9 seconds.

In [13]:
rad = nndc.fetch_decay_radiation(t_range=[1e9, None], type='Gamma', e_range=[1460, 1462])
print(rad[cols])

    Z Element   A Decay Mode Radiation Radiation Energy (keV) Radiation Intensity (%)  Parent Energy Level (MeV)
0  19       K  40         EC         G       1460.822+/-0.006            10.66+/-0.18                        0.0


## Example 7: Find isotopes with lines at 661.7 keV for 120 &le; A &le; 160

This is similar to the last example, but this one shows how to use the ```a_range``` keyword.

In [14]:
rad = nndc.fetch_decay_radiation(a_range=[120, 160], e_range=[661.2, 662.2], i_range=[0.1, None])
rad = rad[rad['Parent Energy Level (MeV)'] == 0]  # ground states only
print(rad[cols])

     Z Element    A Decay Mode Radiation Radiation Energy (keV) Radiation Intensity (%) Parent Energy Level (MeV)
0   53       I  120         EC         G          662.10+/-0.30             1.05+/-0.04                         0
1   54      XE  121         EC        BP                662+/-5             0.48+/-0.08                         0
2   55      CS  137         B-         G      661.6570+/-0.0030            85.10+/-0.20                         0
3   55      CS  143         B-         G          662.10+/-0.10               3.0+/-0.5                         0
5   57      LA  144         B-        BM               662+/-25             0.63+/-0.13                         0
6   61      PM  139         EC        BP               662+/-14           0.110+/-0.030                         0
7   61      PM  139         EC         G            661.3+/-0.4             0.41+/-0.08                         0
8   61      PM  152         B-         G            661.7+/-0.4           0.240+/-0.030 

## Example 8: Find long-lived isotopes with lines at 186 keV
Famously, Ra-226 and U-235 have a similar gamma-ray line here.

In [15]:
rad = nndc.fetch_decay_radiation(e_range=[185, 187], i_range=(1, None), t_range=(1e10, None))
rad = rad[rad['Parent Energy Level (MeV)'] == 0]  # ground states only
print(rad[cols])

    Z Element    A Decay Mode Radiation Radiation Energy (keV) Radiation Intensity (%)  Parent Energy Level (MeV)
0  88      RA  226          A         G        186.211+/-0.013             3.64+/-0.04                        0.0
1  92       U  235          A         G        185.715+/-0.005              57.0+/-0.6                        0.0


## Example 9: Find even- and odd-A isotopes of Hydrogen

You can use the ```a_even``` and ```a_odd``` keywords to select only even or odd ```a``` values. There are equivalents for ```z``` and ```n``` as well.

In [16]:
evens = nndc.fetch_wallet_card(z=1, a_even=True, elevel_range=(0, 0))
print(evens[['Z', 'Element', 'A', 'Abundance (%)', 'T1/2 (s)']])

   Z Element  A  Abundance (%)      T1/2 (s)
0  1       H  2  0.011+/-0.007           inf
1  1       H  4           None           NaN
2  1       H  6           None  2.848000e-22


In [17]:
odds = nndc.fetch_wallet_card(z=1, a_odd=True, elevel_range=(0, 0))
print(odds[['Z', 'Element', 'A', 'Abundance (%)', 'T1/2 (s)']])

   Z Element  A   Abundance (%)      T1/2 (s)
0  1       H  1  99.989+/-0.007           inf
1  1       H  3            None  3.888000e+08
2  1       H  5            None  7.995000e-23


## Example 10: Uranium-238

### 10a: Find all significant U-238 series gamma-ray lines

First we will define a function:

In [18]:
def series_radiation(isotopes, rtype='Gamma', emin=30., imin=0.7, branchings=None):
    """Return a DataFrame of all the gammas from the decay series."""
    df_series = pd.DataFrame()
    if branchings is None:
        branchings = np.ones_like(isotopes)
    assert len(isotopes) == len(branchings)
    for isotope, branching in zip(isotopes, branchings):
        print(isotope, branching)
        isotope_id = isotope
        if isotope.endswith('m'):
            isotope_id = isotope[:-1]
        df = nndc.fetch_decay_radiation(nuc=isotope_id, type=rtype, e_range=(emin, 1e4), i_range=(imin, None))
        if isotope.endswith('m'):
            df = df.loc[df['Parent Energy Level (MeV)'] > 0.]
        else:
            df = df.loc[df['Parent Energy Level (MeV)'] == 0.]
        df['Radiation Intensity (%)'] *= float(branching)
        if len(df) == 0:
            continue
        df_series = df_series.append(df)
    return df_series

Now let's run this function on the U-238 series:

In [19]:
series = ['U-238', 'Th-234', 'Pa-234m', 'U-234',
          'Th-230', 'Ra-226', 'Rn-222', 'Po-218',
          'Pb-214', 'Bi-214', 'Po-214', 'Pb-210',
          'Bi-210', 'Po-210']
gammas = series_radiation(series)
gammas.sort_values(by='Radiation Energy (keV)', inplace=True)
fields = ['Z', 'Element', 'A', 'Decay Mode',
          'Radiation', 'Radiation subtype',
          'Radiation Energy (keV)', 'Radiation Intensity (%)',
          'Daughter']
print(gammas[fields])

U-238 1
Th-234 1
Pa-234m 1
U-234 1
Th-230 1
Ra-226 1
Rn-222 1
Po-218 1
Pb-214 1
Bi-214 1
Po-214 1
Pb-210 1
Bi-210 1
Po-210 1
     Z Element    A Decay Mode Radiation Radiation subtype Radiation Energy (keV) Radiation Intensity (%) Daughter
0   82      PB  210         B-         G                         46.5390+/-0.0010             4.25+/-0.04    210BI
0   82      PB  214         B-         G                         53.2284+/-0.0018           1.075+/-0.007    214BI
0   90      TH  234         B-         G                           63.290+/-0.020               3.7+/-0.4    234PA
1   82      PB  214         B-         G            XR ka2                 74.815             5.80+/-0.30    214BI
2   82      PB  214         B-         G            XR ka1                 77.107               9.7+/-0.5    214BI
0   83      BI  214         B-         G            XR ka1                  79.29           0.907+/-0.023    214PO
3   82      PB  214         B-         G            XR kb3            

### 10b: Find all significant U-238 alpha lines

In [20]:
alphas = series_radiation(series, rtype='Alpha')
alphas.sort_values(by='Radiation Energy (keV)', inplace=True)
print(alphas[fields])

U-238 1
Th-234 1
Pa-234m 1
U-234 1
Th-230 1
Ra-226 1
Rn-222 1
Po-218 1
Pb-214 1
Bi-214 1
Po-214 1
Pb-210 1
Bi-210 1
Po-210 1
    Z Element    A Decay Mode Radiation Radiation subtype Radiation Energy (keV) Radiation Intensity (%) Daughter
0  92       U  238          A         A                                 4151+/-5              21.0+/-3.0    234TH
1  92       U  238          A         A                             4198.0+/-3.0              79.0+/-3.0    234TH
0  88      RA  226          A         A                             4601.0+/-1.0           6.160+/-0.030    222RN
0  90      TH  230          A         A                             4620.5+/-1.5            23.40+/-0.10    226RA
1  90      TH  230          A         A                             4687.0+/-1.5            76.30+/-0.30    226RA
0  92       U  234          A         A                             4722.4+/-1.4            28.42+/-0.09    230TH
1  92       U  234          A         A                             4774.6+/-

### 10c: Find all significant Th-232 series gamma-ray lines

Unlike the U-238 series, we need to provide branching ratios to account for the decay of Bi-212 into either Po-212 or Tl-208.

In [21]:
series = ['Th-232', 'Ra-228', 'Ac-228', 'Th-228',
          'Ra-224', 'Rn-220', 'Po-216', 'Pb-212',
          'Bi-212', 'Po-212', 'Tl-208']
branchings = [1, 1, 1, 1,
              1, 1, 1, 1,
              1, 0.6406, 0.3594]
gammas = series_radiation(series, branchings=branchings)
gammas.sort_values(by='Radiation Energy (keV)', inplace=True)
fields = ['Z', 'Element', 'A', 'Decay Mode',
          'Radiation', 'Radiation subtype',
          'Radiation Energy (keV)', 'Radiation Intensity (%)',
          'Daughter']
print(gammas[fields])

Th-232 1
Ra-228 1
Ac-228 1
Th-228 1
Ra-224 1
Rn-220 1
Po-216 1
Pb-212 1
Bi-212 1
Po-212 0.6406
Tl-208 0.3594
     Z Element    A Decay Mode Radiation Radiation subtype Radiation Energy (keV) Radiation Intensity (%) Daughter
0   83      BI  212          A         G                           39.857+/-0.004             1.06+/-0.09    208TL
0   81      TL  208         B-         G            XR ka2                 72.805           0.722+/-0.022    208PB
0   82      PB  212         B-         G            XR ka2                 74.815            10.28+/-0.25    212BI
1   81      TL  208         B-         G            XR ka1                 74.969           1.204+/-0.032    208PB
1   82      PB  212         B-         G            XR ka1                 77.107              17.1+/-0.4    212BI
0   90      TH  228          A         G                         84.3730+/-0.0030             1.19+/-0.04    224RA
2   81      TL  208         B-         G            XR kb1                 84.938     

### 10d: Find all significant Th-232 alpha lines

In [22]:
alphas = series_radiation(series, branchings=branchings, rtype='Alpha')
alphas.sort_values(by='Radiation Energy (keV)', inplace=True)
print(alphas[fields])

Th-232 1
Ra-228 1
Ac-228 1
Th-228 1
Ra-224 1
Rn-220 1
Po-216 1
Pb-212 1
Bi-212 1
Po-212 0.6406
Tl-208 0.3594
    Z Element    A Decay Mode Radiation Radiation subtype Radiation Energy (keV) Radiation Intensity (%) Daughter
0  90      TH  232          A         A                             3947.2+/-2.0              21.7+/-1.3    228RA
1  90      TH  232          A         A                             4012.3+/-1.4              78.2+/-1.3    228RA
0  90      TH  228          A         A                           5340.36+/-0.15              26.0+/-1.0    224RA
1  90      TH  228          A         A                           5423.15+/-0.22              73.4+/-0.5    224RA
0  88      RA  224          A         A                             5448.6+/-1.2             5.06+/-0.05    220RN
1  88      RA  224          A         A                           5685.37+/-0.15            94.92+/-0.05    220RN
3  83      BI  212          A         A                         6050.780+/-0.030            2